<a href="https://colab.research.google.com/github/tanyaoley/colab/blob/master/06_02_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 6.2 Классификация.Фильмы

Для тех, кто хочет поставить точку в рекомендательных системах.

### Рекомендательная система и kNN

В прошлый раз мы использовали kMeans, тоже метрический алгоритм, но кластеризации, для построения "рекомендательной ситсемы". На самом деле тогда вы строили кластеры жанров/фильмов на основе предпочтений пользователей.  

Теперь попробуем kNN для рекомендаций. Мы построим модель, в которой не будет кластеров, но которая сможет выводить список наиболее похожих фильмов для заданного.

Подготовка датасета похожа на то, что мы уже делали. Отличаться будет только последний пункт, когда мы запустим kNN с косинусной метрикой.

Будем использовать известный нам [датасет с оценками фильмов](http://www.cs.umn.edu/GroupLens). Только в этот раз мы не будем кластеризовывать жанры, а будем рекомендовать похожие фильмы, основываясь на оценках других пользователей. То есть мы будем по факту предсказывать предпочтение нового пользователя на основе других.

Для рекомендательной системы в датасете в объектами будут пользователи, а признаками - оценки фильма данным пользователем.

![](https://i.imgur.com/hn3xjZl.png)

Мы будем использовать метрический алгоритм для поиска ближайших соседей, тем самым рекомендовать фильмы близкие к данному.

In [0]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
colab = True 
if colab:
    from google.colab import drive
    drive.mount('/content/drive/')
    movies = pd.read_csv('/content/drive/My Drive/Data/imdb_movies.csv.gz', index_col='movieId')
    ratings = pd.read_csv('/content/drive/My Drive/Data/imdb_ratings.csv.gz')
else:
    movies = pd.read_csv('../../data/imdb_movies.csv.gz', index_col='movieId')
    ratings = pd.read_csv('../../data/imdb_ratings.csv.gz')
print(movies.shape)
print(ratings.shape)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/
(9125, 2)
(100004, 4)


In [3]:
movies.head()

,title,genres
movieId,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,Jumanji (1995),Adventure|Children|Fantasy
3,Grumpier Old Men (1995),Comedy|Romance
4,Waiting to Exhale (1995),Comedy|Drama|Romance
5,Father of the Bride Part II (1995),Comedy


In [4]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


### 1.1 Поиск популярных фильмов (3 балла)

Среди всех фильмов есть не популярные, для которых выставлена 1-2 оценки. Такого количества данных не достаточно, поэтому нужно срезать много фильмов с маленьким числом отзывов.

Посчтитайте количество оценок для каждого фильма.

In [0]:
ratings_counts = pd.DataFrame(ratings['movieId'].value_counts(),index = ratings['movieId'].unique())

Выведите сводную статистику по количеству оценок фильмов. (общее число фильмов, среднее количество оценок, дисперсию, квантили)

In [6]:
print('total film number: {}'.format(ratings_counts.shape[0]))
print(ratings_counts.mean()[0])
print(ratings_counts.std()[0])
#квантили
print(np.quantile(ratings_counts,q = [0,0.25,0.5,0.75,1], axis=0))

total film number: 9066
11.030664019413193
24.050799967666727
[[  1.]
 [  1.]
 [  3.]
 [  9.]
 [341.]]


Как мы видим `75%` квантиль показывет, что `75%` фильмов имело очень мало оценок. Нам столько будет недостаточно. Посчтитайте квантили в диапазоне от `75%` до `100%` с шагом в `1%`. Выберите тот квантиль, при котором количество оценок равно 20. Напишите какой это квантиль. Напишите сколько всего фильмов останется, если отбросить все фильмы у которых рейтинг меньше этого квантиля.

In [7]:
quan_20 = 0
for i in range (75,100):
    res = np.quantile(ratings_counts,q = i/100, axis=0)
    print(res)
    if res == 20:
        quan_20 = i
print('квантиль, при котором количество оценок равно 20 {}'.format(quan_20))
sorted_ratings = ratings_counts[ratings_counts['movieId'] > 20]
print(sorted_ratings.shape[0])

[9.]
[10.]
[10.]
[11.]
[12.]
[13.]
[14.]
[15.]
[16.]
[17.]
[19.]
[20.]
[22.]
[23.]
[25.]
[28.]
[31.]
[34.]
[38.]
[44.]
[49.]
[58.]
[69.05]
[89.]
[123.]
квантиль, при котором количество оценок равно 20 86
1247


### 1.2 Соединяем таблицы (1 балл)

Будет немного не удобно искать фильмы по их идентификаторам, нам нужны названия. Поэтому соедините две таблицы `ratings` и `movies` по `movieId` и отфильтруйте отзывы на редкие фильмы, которые мы нашли в предыдущем шаге. Выведите первые 5 элементов полученного датасета.

In [15]:
from collections import Counter
c = ratings.join(movies, on =['movieId'])
count= Counter(ratings_movies['movieId'])
delete = list()
for i in range(len(ratings_movies)):
  if count[ratings_movies.iloc[i]['movieId']]<=20:
    delete.append(i)
ratings_movies =ratings_movies.drop(ratings_movies.index[delete])
ratings_movies.head(5) 
 
 

,userId,movieId,rating,timestamp,title,genres
0,1,31,2.5,1260759144,Dangerous Minds (1995),Drama
1,1,1029,3.0,1260759179,Dumbo (1941),Animation|Children|Drama|Musical
2,1,1061,3.0,1260759182,Sleepers (1996),Thriller
3,1,1129,2.0,1260759185,Escape from New York (1981),Action|Adventure|Sci-Fi|Thriller
4,1,1172,4.0,1260759205,Cinema Paradiso (Nuovo cinema Paradiso) (1989),Drama


### 1.3 Матрица Фильмы-Пользователи

В нашем случае мы будем искать похожие фильмы, поэтому сделайте из датасета оценок матрицу, где в строках фильмы, в колонках пользователи, а в ячейках оценки. Пропущенные значение, то есть отсутсвие оценки, заполните 0.

Для увеличения скорости выполнения алгоритма ближайших соседей примените функцию [`scipy.sparse.csc_matrix`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.html),  которая создаст плотное представление нашей разреженой матрицы.

In [0]:
movies_ratings = pd.pivot_table(ratings_movies, values = 'rating', index=['title'],
              columns=['userId'], aggfunc=np.sum).fillna(0)

In [0]:
from scipy.sparse import csr_matrix
sp_matr = csr_matrix(movies_ratings)

### 1.4 Тренировка модели рекомендаций 

Так как по сути у нас нет каких-то меток классов, мы будем использовать алгоритма без учителя [`sklearn.neighbors.NearestNeighbors`](http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.NearestNeighbors.html). Научите его по данным нашей матрицы пользователей-фильмов и напишите **функцию**, принимающую на вход название фильма и возвращающую список из 6 рекомендованых фильмов, начиная от более похожего.

Для определения "похожести" двух фильмов мы будем использовать косинусную меру, вместо евклидова или минковского:

$$\LARGE cos(\overrightarrow{x},\overrightarrow{y}) = \frac{\overrightarrow{x} \cdot \overrightarrow{y}}{||\overrightarrow{x}|| \times ||\overrightarrow{y}||}$$

In [62]:
from sklearn.neighbors import NearestNeighbors
neigh = NearestNeighbors(6+1, metric ='cosine')
neigh.fit(sp_matr)

NearestNeighbors(algorithm='auto', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=None, n_neighbors=7, p=2,
                 radius=1.0)

In [0]:
def similar(name='Pulp Fiction (1994)', n=6):
    return movies_ratings.index[neigh.kneighbors(movies_ratings[movies_ratings.index == name],n+1)[1]]
    

Выведите рекомендации для фильмов:
- `Pulp Fiction (1994)`
- `Star Wars: Episode V - The Empire Strikes Back (1980)`
- `Lord of the Rings: The Two Towers, The (2002)`

In [64]:
similar('Pulp Fiction (1994)')

Index([['Pulp Fiction (1994)', 'Silence of the Lambs, The (1991)', 'Shawshank Redemption, The (1994)', 'Seven (a.k.a. Se7en) (1995)', 'Forrest Gump (1994)', 'Usual Suspects, The (1995)', 'Jurassic Park (1993)']], dtype='object', name='title')

In [65]:
similar('Star Wars: Episode V - The Empire Strikes Back (1980)')

Index([['Star Wars: Episode V - The Empire Strikes Back (1980)', 'Star Wars: Episode IV - A New Hope (1977)', 'Star Wars: Episode VI - Return of the Jedi (1983)', 'Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)', 'Indiana Jones and the Last Crusade (1989)', 'Back to the Future (1985)', 'Terminator, The (1984)']], dtype='object', name='title')

In [66]:
similar('Lord of the Rings: The Two Towers, The (2002)')

Index([['Lord of the Rings: The Two Towers, The (2002)', 'Lord of the Rings: The Return of the King, The (2003)', 'Lord of the Rings: The Fellowship of the Ring, The (2001)', 'Pirates of the Caribbean: The Curse of the Black Pearl (2003)', 'Matrix, The (1999)', 'Spider-Man (2002)', 'Shrek (2001)']], dtype='object', name='title')

### 1.5 Как посчитать качество рекомендации?

Как посчитать качество нашей классификации? У нас нет учителя, нет отложенной выборки. Мы просто посчитали все рейтинги и на основе них сделали предположения о похожести товаров. 

А как вы думаете можно проверить качество нашей рекомендательной системы? Напишите развернутый ответ своими словами.

возможно для проверки качества системы стоит оценивать по расстояниям до ближайшего соседа, то есть чем ближе расположены точки, тем больше похожи фильмы
можно брать список просмотренного у пользователя и найти самые близкие к фильмам по расстояниям(то есть самое похожеее к тому, что пользователь смотрел)



![](https://69.media.tumblr.com/06a071083b3e62cd76f31af07ecb895f/tumblr_p49r4rWXG21uxovwqo1_540.gif)